In [1]:
import boto3
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

# I have removed Access Key and Id which I have used for fetching the data from s3 bucket

In [3]:
s3 = boto3.client(
    service_name='s3',
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

 - Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

In [4]:
FILE_NAME_COUNT = '/pub/time.series/pr/pr.data.0.Current'
FILE_NAME_PART_POPULATION = 'task2/population_data.json'

In [7]:
response_count = s3.get_object(Bucket=S3_BUCKET, Key=FILE_NAME_COUNT)
file_content_count = response_count['Body'].read().decode('utf-8') 
df_pandas_count = pd.read_csv(StringIO(file_content_count), delimiter='\t')

In [8]:
df_pandas_count.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [9]:
import json
response_population = s3.get_object(Bucket=S3_BUCKET, Key=FILE_NAME_PART_POPULATION)
file_content_population = response_population['Body'].read().decode('utf-8')
df_pandas_population = json.loads(file_content_population)

In [10]:
 print(df_pandas_population)

{'data': [{'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2022, 'Year': '2022', 'Population': 331097593, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2021, 'Year': '2021', 'Population': 329725481, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2020, 'Year': '2020', 'Population': 326569308, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2019, 'Year': '2019', 'Population': 324697795, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2018, 'Year': '2018', 'Population': 322903030, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2017, 'Year': '2017', 'Population': 321004407, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2016, 'Year': '2016', 'Population': 318558162, 'Slug Nation': 'united

In [11]:
data = df_pandas_population['data']

In [12]:
data

[{'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2022,
  'Year': '2022',
  'Population': 331097593,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2021,
  'Year': '2021',
  'Population': 329725481,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2020,
  'Year': '2020',
  'Population': 326569308,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2019,
  'Year': '2019',
  'Population': 324697795,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2018,
  'Year': '2018',
  'Population': 322903030,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2017,
  'Year': '2017',
  'Population': 321004407,
  'Slug Nation': 'united-states'},
 {'ID Nation': '01000US',
  'Nation': 'United States',
  'ID Year': 2016,
  

In [13]:
df_pandas_population = pd.DataFrame(data)

In [14]:
print(df_pandas_population.head())

  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2022  2022   331097593  united-states
1   01000US  United States     2021  2021   329725481  united-states
2   01000US  United States     2020  2020   326569308  united-states
3   01000US  United States     2019  2019   324697795  united-states
4   01000US  United States     2018  2018   322903030  united-states


In [15]:
spark = SparkSession.builder.appName("S3FileProcessing").getOrCreate()

In [38]:
df_count_spark = spark.createDataFrame(df_pandas_count)
df_spark_population = spark.createDataFrame(df_pandas_population)

In [39]:
df_count_spark.show()

+-----------------+----+------+------------+--------------+
|series_id        |year|period|       value|footnote_codes|
+-----------------+----+------+------------+--------------+
|PRS30006011      |1995|   Q01|         2.6|           NaN|
|PRS30006011      |1995|   Q02|         2.1|           NaN|
|PRS30006011      |1995|   Q03|         0.9|           NaN|
|PRS30006011      |1995|   Q04|         0.1|           NaN|
|PRS30006011      |1995|   Q05|         1.4|           NaN|
|PRS30006011      |1996|   Q01|        -0.2|           NaN|
|PRS30006011      |1996|   Q02|        -0.3|           NaN|
|PRS30006011      |1996|   Q03|        -0.1|           NaN|
|PRS30006011      |1996|   Q04|         0.2|           NaN|
|PRS30006011      |1996|   Q05|        -0.1|           NaN|
|PRS30006011      |1997|   Q01|         0.3|           NaN|
|PRS30006011      |1997|   Q02|         0.7|           NaN|
|PRS30006011      |1997|   Q03|         1.0|           NaN|
|PRS30006011      |1997|   Q04|         

In [19]:
df_spark_population.show()

+---------+-------------+-------+----+----------+-------------+
|ID Nation|       Nation|ID Year|Year|Population|  Slug Nation|
+---------+-------------+-------+----+----------+-------------+
|  01000US|United States|   2022|2022| 331097593|united-states|
|  01000US|United States|   2021|2021| 329725481|united-states|
|  01000US|United States|   2020|2020| 326569308|united-states|
|  01000US|United States|   2019|2019| 324697795|united-states|
|  01000US|United States|   2018|2018| 322903030|united-states|
|  01000US|United States|   2017|2017| 321004407|united-states|
|  01000US|United States|   2016|2016| 318558162|united-states|
|  01000US|United States|   2015|2015| 316515021|united-states|
|  01000US|United States|   2014|2014| 314107084|united-states|
|  01000US|United States|   2013|2013| 311536594|united-states|
+---------+-------------+-------+----+----------+-------------+



In [20]:
df_spark_population.printSchema()

root
 |-- ID Nation: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- ID Year: long (nullable = true)
 |-- Year: string (nullable = true)
 |-- Population: long (nullable = true)
 |-- Slug Nation: string (nullable = true)



In [21]:
new_pop = df_spark_population.selectExpr(" 'ID Nation' as id_nation", "Nation as nation",
                               " 'ID Year' as id_year",
                               "cast(Year as int) as year",
                               "Population as population",
                               " 'Slug Nation' as slug_nation"
                               )

In [22]:
new_pop.printSchema()

root
 |-- id_nation: string (nullable = false)
 |-- nation: string (nullable = true)
 |-- id_year: string (nullable = false)
 |-- year: integer (nullable = true)
 |-- population: long (nullable = true)
 |-- slug_nation: string (nullable = false)



- Using the dataframe from the population data API (Part 2), generate the 
mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.
from pyspark.sql.functions import mean, stddev

In [29]:
filtered_data = new_pop.filter(new_pop['year'].between(2013, 2018))

statistics = filtered_data.select(
    mean('population').alias('mean_population'),
    stddev('population').alias('stddev_population')
)

In [30]:
statistics.show()

+---------------+-----------------+
|mean_population|stddev_population|
+---------------+-----------------+
|   3.17437383E8|4257089.541529327|
+---------------+-----------------+



 - Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [40]:
df_count_spark.show()

+-----------------+----+------+------------+--------------+
|series_id        |year|period|       value|footnote_codes|
+-----------------+----+------+------------+--------------+
|PRS30006011      |1995|   Q01|         2.6|           NaN|
|PRS30006011      |1995|   Q02|         2.1|           NaN|
|PRS30006011      |1995|   Q03|         0.9|           NaN|
|PRS30006011      |1995|   Q04|         0.1|           NaN|
|PRS30006011      |1995|   Q05|         1.4|           NaN|
|PRS30006011      |1996|   Q01|        -0.2|           NaN|
|PRS30006011      |1996|   Q02|        -0.3|           NaN|
|PRS30006011      |1996|   Q03|        -0.1|           NaN|
|PRS30006011      |1996|   Q04|         0.2|           NaN|
|PRS30006011      |1996|   Q05|        -0.1|           NaN|
|PRS30006011      |1997|   Q01|         0.3|           NaN|
|PRS30006011      |1997|   Q02|         0.7|           NaN|
|PRS30006011      |1997|   Q03|         1.0|           NaN|
|PRS30006011      |1997|   Q04|         

In [49]:
df_count_spark.printSchema()

root
 |-- series_id        : string (nullable = true)
 |-- year: long (nullable = true)
 |-- period: string (nullable = true)
 |--        value: double (nullable = true)
 |-- footnote_codes: string (nullable = true)



In [64]:
from pyspark.sql.functions import col

cleaned_df = df_count_spark.select(
    [col(col_name).alias(col_name.strip()) for col_name in df_count_spark.columns]
)

cleaned_df.printSchema()

root
 |-- series_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- period: string (nullable = true)
 |-- value: double (nullable = true)
 |-- footnote_codes: string (nullable = true)



In [65]:
cleaned_df.show()

+-----------------+----+------+-----+--------------+
|        series_id|year|period|value|footnote_codes|
+-----------------+----+------+-----+--------------+
|PRS30006011      |1995|   Q01|  2.6|           NaN|
|PRS30006011      |1995|   Q02|  2.1|           NaN|
|PRS30006011      |1995|   Q03|  0.9|           NaN|
|PRS30006011      |1995|   Q04|  0.1|           NaN|
|PRS30006011      |1995|   Q05|  1.4|           NaN|
|PRS30006011      |1996|   Q01| -0.2|           NaN|
|PRS30006011      |1996|   Q02| -0.3|           NaN|
|PRS30006011      |1996|   Q03| -0.1|           NaN|
|PRS30006011      |1996|   Q04|  0.2|           NaN|
|PRS30006011      |1996|   Q05| -0.1|           NaN|
|PRS30006011      |1997|   Q01|  0.3|           NaN|
|PRS30006011      |1997|   Q02|  0.7|           NaN|
|PRS30006011      |1997|   Q03|  1.0|           NaN|
|PRS30006011      |1997|   Q04|  1.5|           NaN|
|PRS30006011      |1997|   Q05|  0.9|           NaN|
|PRS30006011      |1998|   Q01|  1.6|         

In [81]:
best_year = cleaned_df.selectExpr(
    "series_id",
    "year",
    "sum(value) over (partition by series_id, year) as yearly_sum",
    "rank() over (partition by series_id order by sum(value) over (partition by series_id, year) desc) as rank"
).distinct().where("rank == 1").select("series_id", "year", "yearly_sum")

In [82]:
best_year.show()

+-----------------+----+------------------+
|        series_id|year|        yearly_sum|
+-----------------+----+------------------+
|PRS30006011      |2022|              20.5|
|PRS30006012      |2022|17.099999999999998|
|PRS30006013      |1998|           704.125|
|PRS30006021      |2010|17.599999999999998|
|PRS30006022      |2010|              12.5|
|PRS30006023      |2014|           503.171|
|PRS30006031      |2022|20.400000000000002|
|PRS30006032      |2021|17.099999999999998|
|PRS30006033      |1998|           700.712|
|PRS30006061      |2022|              37.1|
|PRS30006062      |2021|              31.6|
|PRS30006063      |2023| 629.1310000000001|
|PRS30006081      |2021|              23.3|
|PRS30006082      |2021|              23.3|
|PRS30006083      |2021|           112.341|
|PRS30006091      |2002|43.400000000000006|
|PRS30006092      |2002|              44.5|
|PRS30006093      |2011|           520.086|
|PRS30006101      |2020|              33.4|
|PRS30006102      |2020|        

 - Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset). The below table shows an example of one row that might appear in the resulting table:

In [85]:
cleaned_df.printSchema()

root
 |-- series_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- period: string (nullable = true)
 |-- value: double (nullable = true)
 |-- footnote_codes: string (nullable = true)



In [95]:
df_spark_population.printSchema()

root
 |-- ID Nation: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- ID Year: long (nullable = true)
 |-- Year: string (nullable = true)
 |-- Population: long (nullable = true)
 |-- Slug Nation: string (nullable = true)



In [121]:
df_spark_population = df_spark_population.withColumnRenamed("Year", "year")

In [124]:
df_spark_population.printSchema()

root
 |-- ID Nation: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- ID Year: long (nullable = true)
 |-- year: string (nullable = true)
 |-- Population: long (nullable = true)
 |-- Slug Nation: string (nullable = true)



In [133]:
df_all_data = cleaned_df.join(df_spark_population, 'year')

In [143]:
df_all_data.selectExpr("series_id","year","period","value","Population").where("trim(series_id) = 'PRS30006032' and trim(period) = 'Q01'").show()

+-----------------+----+------+-----+----------+
|        series_id|year|period|value|Population|
+-----------------+----+------+-----+----------+
|PRS30006032      |2013|   Q01|  0.8| 311536594|
|PRS30006032      |2014|   Q01| -0.1| 314107084|
|PRS30006032      |2015|   Q01| -1.6| 316515021|
|PRS30006032      |2016|   Q01| -1.4| 318558162|
|PRS30006032      |2017|   Q01|  0.7| 321004407|
|PRS30006032      |2018|   Q01|  0.4| 322903030|
|PRS30006032      |2019|   Q01| -1.6| 324697795|
|PRS30006032      |2020|   Q01| -6.7| 326569308|
|PRS30006032      |2021|   Q01|  1.2| 329725481|
|PRS30006032      |2022|   Q01|  5.6| 331097593|
+-----------------+----+------+-----+----------+

